In [1]:
import torch
import torchvision.models as models
from torch import nn
from torch.optim import lr_scheduler
import torchvision
from PIL import Image
from torchvision import transforms

## Importing Model

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model = models.resnet152(weights=None)
num_ftrs = model.fc.in_features
out_ftrs = 5
model.fc = nn.Sequential(nn.Linear(num_ftrs, 512),nn.ReLU(),nn.Linear(512,out_ftrs),nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(filter(lambda p:p.requires_grad,model.parameters()) , lr = 0.00001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
model.to(device);

In [4]:
for name,child in model.named_children():
    if name in ['layer2','layer3','layer4','fc']:
        #print(name + 'is unfrozen')
        for param in child.parameters():
            param.requires_grad = True
    else:
        #print(name + 'is frozen')
        for param in child.parameters():
            param.requires_grad = False

In [5]:
optimizer = torch.optim.Adam(filter(lambda p:p.requires_grad,model.parameters()) , lr = 0.000001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [6]:
checkpoint = torch.load(f"C:\\Users\\plm\\Downloads\\aptos2019-blindness-detection\\classifier.pt",map_location='cpu')

In [7]:
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [8]:
classes = ['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative DR']
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [9]:
def inference(model, file, transform, classes):
    file = Image.open(file).convert('RGB')
    img = transform(file).unsqueeze(0)
    print('Transforming your image...')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    with torch.no_grad():
        print('Passing your image to the model....')
        out = model(img.to(device))
        ps = torch.exp(out)
        top_p, top_class = ps.topk(1, dim=1)
        value = top_class.item()
        print("Predicted Severity Value: ", value)
        print("class is: ", classes[value])
        return value, classes[value]

In [10]:
# x, y = inference(model, "C:\\Users\\plm\Downloads\\aptos2019-blindness-detection\\test_images\\fdde61dd1bde.png", test_transforms, classes)

In [11]:
# x, y

## Interface

In [12]:
def predict(image):
    img = test_transforms(image).unsqueeze(0)
    print('Transforming your image...')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    with torch.no_grad():
        print('Passing your image to the model....')
        out = model(img.to(device))
        ps = torch.exp(out)
        top_p, top_class = ps.topk(1, dim=1)
        value = top_class.item()
        print("Predicted Severity Value: ", value)
        print("class is: ", classes[value])
        print('Your image is printed:')
        return classes[value]

In [13]:
import gradio as gr

C:\Users\plm\miniconda3\envs\EPICS_pr2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
gr.Interface(fn=predict,
             inputs=gr.Image(type="pil"),
             outputs=gr.Label(num_top_classes=5),
             examples=[f"C:\\Users\\plm\\Downloads\\aptos2019-blindness-detection\\train_images\\ff8a0b45c789.png", 
                       f"C:\\Users\\plm\\Downloads\\aptos2019-blindness-detection\\train_images\\ffc04fed30e6.png",
                       f"C:\\Users\\plm\\Downloads\\aptos2019-blindness-detection\\train_images\\fecf4c5ae84b.png",
                       f"C:\\Users\\plm\\Downloads\\aptos2019-blindness-detection\\train_images\\0bf37ca3156a.png",
                       f"C:\\Users\\plm\\Downloads\\aptos2019-blindness-detection\\train_images\\f6f433f3306f.png"
                      ]).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Transforming your image...
Passing your image to the model....
Predicted Severity Value:  2
class is:  Moderate
Your image is printed:
Transforming your image...
Passing your image to the model....
Predicted Severity Value:  3
class is:  Severe
Your image is printed:


## Aaditya End